## Notebook controller to control the Simulation


In [ ]:
import time
import socket
import pickle

from flax import serialization

from simulationsandbox.utils.network import SERVER

In [ ]:
PORT = 5050
ADDR = (SERVER, PORT)
DATA_SIZE = 40000
EVAL_TIME = 10

In [ ]:
# Start the server and intialize connection

def connect_client():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    print(f"Connected to {ADDR}")

    sim_type = client.recv(1024).decode()
    print(f"{sim_type = }")
    state_example = pickle.loads(client.recv(DATA_SIZE))
    state_bytes_size = len(serialization.to_bytes(state_example))
    response = "NOTEBOOK"
    client.send(response.encode())
    time.sleep(1)

    return client, state_example, state_bytes_size

In [ ]:
def close_client(client):
    client.send("CLOSE_CONNECTION".encode())

def get_state(client, state_example, state_bytes_size):
    client.send("GET_STATE".encode())
    response = client.recv(state_bytes_size)
    return serialization.from_bytes(state_example, response)

def pause(client):
    client.send("PAUSE".encode())

def resume(client):
    client.send("RESUME".encode())

def stop(client):
    client.send("STOP".encode())

def start(client):
    client.send("START".encode())

In [ ]:
client, state_example, state_bytes_size = connect_client()

In [ ]:
pause(client)

In [ ]:
resume(client)

In [ ]:
state = get_state(client, state_example, state_bytes_size)

In [ ]:
close_client(client)